<a href="https://colab.research.google.com/github/snvssk/data298A/blob/saidev/SolarModel_SVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import seaborn as sns
from glob import glob
import numpy as np

from datetime import datetime
from random import randint
from math import sqrt
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVR


In [2]:
from google.colab import drive
# drive.mount('/content/drive')

In [3]:
df= pd.read_csv('/content/drive/MyDrive/final_weather.csv')
df

,Year,Month,Day,Hour,GHI,Cloud_Type,Dew_Point,Solar_Zenith_Angle,Surface_Albedo,Wind_Speed,Precipitable_Water,Wind_Direction,Relative_Humidity,Temperature,Pressure
0,2016,1,1,0.0,0,1,0.0,165.44,0.099,1.0,0.705,72.0,87.98,4.0,1020
1,2016,1,1,0.5,0,1,0.0,165.13,0.099,1.0,0.714,72.0,87.97,3.0,1020
2,2016,1,1,1.0,0,1,0.0,162.29,0.099,1.0,0.723,73.1,87.91,3.0,1020
3,2016,1,1,1.5,0,1,0.0,157.90,0.099,1.0,0.730,73.1,87.91,3.0,1020
4,2016,1,1,2.0,0,1,0.0,152.70,0.099,1.0,0.737,65.4,87.69,3.0,1020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526171,2020,12,31,21.5,0,7,7.5,142.70,0.110,1.1,0.900,323.0,83.23,10.2,1006
526172,2020,12,31,22.0,0,7,7.8,148.49,0.110,1.4,1.000,294.0,88.11,9.7,1006
526173,2020,12,31,22.5,0,7,7.8,154.04,0.110,1.6,1.000,298.0,91.73,9.1,1006
526174,2020,12,31,23.0,0,7,8.5,159.12,0.110,1.7,1.000,302.0,99.42,8.6,1006


In [4]:
df_updated = df.copy()
df_updated.isnull().sum()
df_updated = df_updated.dropna()
df_updated

,Year,Month,Day,Hour,GHI,Cloud_Type,Dew_Point,Solar_Zenith_Angle,Surface_Albedo,Wind_Speed,Precipitable_Water,Wind_Direction,Relative_Humidity,Temperature,Pressure
0,2016,1,1,0.0,0,1,0.0,165.44,0.099,1.0,0.705,72.0,87.98,4.0,1020
1,2016,1,1,0.5,0,1,0.0,165.13,0.099,1.0,0.714,72.0,87.97,3.0,1020
2,2016,1,1,1.0,0,1,0.0,162.29,0.099,1.0,0.723,73.1,87.91,3.0,1020
3,2016,1,1,1.5,0,1,0.0,157.90,0.099,1.0,0.730,73.1,87.91,3.0,1020
4,2016,1,1,2.0,0,1,0.0,152.70,0.099,1.0,0.737,65.4,87.69,3.0,1020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526171,2020,12,31,21.5,0,7,7.5,142.70,0.110,1.1,0.900,323.0,83.23,10.2,1006
526172,2020,12,31,22.0,0,7,7.8,148.49,0.110,1.4,1.000,294.0,88.11,9.7,1006
526173,2020,12,31,22.5,0,7,7.8,154.04,0.110,1.6,1.000,298.0,91.73,9.1,1006
526174,2020,12,31,23.0,0,7,8.5,159.12,0.110,1.7,1.000,302.0,99.42,8.6,1006


In [5]:
df_updated['Date'] = df_updated.apply(lambda row: datetime.strptime(f"{int(row.Year)}-{int(row.Month)}-{int(row.Day)}", '%Y-%m-%d'), axis=1)
df_updated['Date'] = pd.to_datetime(df_updated['Date'])
df_updated = df_updated.resample('D', on='Date').mean()
df_updated

,Year,Month,Day,Hour,GHI,Cloud_Type,Dew_Point,Solar_Zenith_Angle,Surface_Albedo,Wind_Speed,Precipitable_Water,Wind_Direction,Relative_Humidity,Temperature,Pressure
Date,,,,,,,,,,,,,,,
2016-01-01,2016.0,1.0,1.0,11.75,121.597222,3.385417,0.784722,107.211875,0.111667,1.070833,0.823941,74.762500,81.707917,5.159722,1015.000000
2016-01-02,2016.0,1.0,2.0,11.75,69.097222,5.850694,3.777778,107.150903,0.112333,1.364583,1.549521,137.237500,86.207292,7.774306,1011.493056
2016-01-03,2016.0,1.0,3.0,11.75,78.434028,7.631944,7.020833,107.084306,0.112333,1.316667,1.862205,101.100000,93.414861,9.815972,1006.041667
2016-01-04,2016.0,1.0,4.0,11.75,60.690972,6.253472,9.125000,107.012083,0.112333,2.756250,2.156691,154.650000,94.380903,11.777778,1002.048611
2016-01-05,2016.0,1.0,5.0,11.75,41.961806,6.128472,8.204861,106.934792,0.112333,2.683333,1.796889,213.112500,96.394167,10.600694,1001.423611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,2020.0,12.0,27.0,11.75,114.125000,5.006944,6.546875,107.385660,0.125000,1.585069,1.115972,150.343750,81.135660,10.082639,1011.857639
2020-12-28,2020.0,12.0,28.0,11.75,96.440972,2.673611,6.453125,107.347986,0.125000,1.460764,1.547222,168.951389,83.628924,9.312847,1009.586806
2020-12-29,2020.0,12.0,29.0,11.75,126.677083,0.888889,4.521875,107.303333,0.124444,1.237153,0.745486,235.479167,76.137882,9.256250,1022.548611


In [29]:
target = df_updated['GHI'] #keeping only target(ghi) data here
input= df_updated.drop(['GHI'], axis=1)
input

,Year,Month,Day,Hour,Cloud_Type,Dew_Point,Solar_Zenith_Angle,Surface_Albedo,Wind_Speed,Precipitable_Water,Wind_Direction,Relative_Humidity,Temperature,Pressure
Date,,,,,,,,,,,,,,
2016-01-01,2016.0,1.0,1.0,11.75,3.385417,0.784722,107.211875,0.111667,1.070833,0.823941,74.762500,81.707917,5.159722,1015.000000
2016-01-02,2016.0,1.0,2.0,11.75,5.850694,3.777778,107.150903,0.112333,1.364583,1.549521,137.237500,86.207292,7.774306,1011.493056
2016-01-03,2016.0,1.0,3.0,11.75,7.631944,7.020833,107.084306,0.112333,1.316667,1.862205,101.100000,93.414861,9.815972,1006.041667
2016-01-04,2016.0,1.0,4.0,11.75,6.253472,9.125000,107.012083,0.112333,2.756250,2.156691,154.650000,94.380903,11.777778,1002.048611
2016-01-05,2016.0,1.0,5.0,11.75,6.128472,8.204861,106.934792,0.112333,2.683333,1.796889,213.112500,96.394167,10.600694,1001.423611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,2020.0,12.0,27.0,11.75,5.006944,6.546875,107.385660,0.125000,1.585069,1.115972,150.343750,81.135660,10.082639,1011.857639
2020-12-28,2020.0,12.0,28.0,11.75,2.673611,6.453125,107.347986,0.125000,1.460764,1.547222,168.951389,83.628924,9.312847,1009.586806
2020-12-29,2020.0,12.0,29.0,11.75,0.888889,4.521875,107.303333,0.124444,1.237153,0.745486,235.479167,76.137882,9.256250,1022.548611


In [7]:
# Normalize the dataset

# sc = StandardScaler()

input = StandardScaler().fit_transform(input)
# target = StandardScaler().fit_transform(target)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(input, target, test_size=0.3, random_state=42)

In [9]:
X_train

array([[ 1.41344013, -1.60067642, -0.65153714, ...,  0.88627732,
        -1.27334155,  2.0511929 ],
       [ 0.70672007, -1.60067642,  0.82533427, ..., -0.24511338,
        -1.15148971,  2.73951816],
       [-0.70672007,  1.58845751,  0.03009582, ..., -0.90395559,
        -1.00143067, -1.13426289],
       ...,
       [ 0.        , -0.44099135, -0.65153714, ...,  0.28054523,
        -0.14602913, -0.05848856],
       [ 0.70672007,  1.58845751,  1.62057273, ...,  0.89167097,
        -1.21749618,  0.90729565],
       [ 0.70672007, -1.60067642,  1.73417822, ...,  0.9628398 ,
        -0.59523077, -0.58644113]])

In [10]:
print("X_train shape",X_train.shape)
print("X_test shape",X_test.shape)
print("y_train shape",y_train.shape)
print("y_text shape",y_test.shape)

X_train shape (1278, 14)
X_test shape (549, 14)
y_train shape (1278, 1)
y_text shape (549, 1)


In [11]:
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR()

In [12]:
y_pred = regressor.predict(X_test)
y_pred

array([265.29358721, 318.97037589, 253.09560275, 151.03984725,
       164.76352259, 133.34011519, 278.90325463, 183.78399442,
       216.06062862, 218.88686916, 296.02299337, 278.4335735 ,
       244.68876101, 287.9700323 , 152.95092904, 138.21227393,
       205.44462747, 132.16160017, 274.2040846 , 167.12723481,
       302.99985132, 239.96762656, 137.47545517, 293.47729994,
       240.1371439 , 285.56168362, 311.34620984, 234.29970968,
       161.46422619, 186.52374944, 288.45329742, 298.31230459,
       205.16086415, 314.64111683, 233.73981401, 207.2493415 ,
       160.75693627, 187.77380277, 151.73890797, 312.02095109,
       219.65154945, 159.92460797, 154.41238305, 140.56151682,
       262.15457886, 283.34444389, 176.27029016, 224.6886688 ,
       179.88588198, 152.68620076, 327.26765814, 303.51208162,
       325.26272325, 325.17225608, 299.14207893, 267.34140337,
       151.25123917, 201.58087768, 154.66142612, 304.80157087,
       165.72812804, 299.67969755, 267.02514446, 105.74

In [13]:

MSE = mean_squared_error(y_test, y_pred)
RMSE = math.sqrt(MSE)

In [14]:
r2_score(y_test, y_pred)

0.7643107393522827

In [15]:
MSE

2357.2285545846057

In [16]:
RMSE

48.55129817610035

In [17]:
# Parameters for tuning

parameters = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001,0.0001],'kernel': ['rbf', 'poly', 'sigmoid']}

In [18]:
grid = GridSearchCV(SVR(),
                    scoring = 'neg_mean_squared_error',
                    param_grid = parameters,
                    refit=True,
                    verbose=2)


In [21]:
grid.fit(X_train,y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.1s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.1s


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.1s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.1s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.1s


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   0.6s


...

[CV] END ................C=100, gamma=0.0001, kernel=sigmoid; total time=   0.1s
[CV] END ................C=100, gamma=0.0001, kernel=sigmoid; total time=   0.1s
[CV] END ................C=100, gamma=0.0001, kernel=sigmoid; total time=   0.1s


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] END ................C=100, gamma=0.0001, kernel=sigmoid; total time=   0.1s


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'poly', 'sigmoid']},
             scoring='neg_mean_squared_error', verbose=2)

In [22]:
grid_predictions = grid.predict(X_test)

In [31]:
print("Best parameters:", grid.best_params_)

Best parameters: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}


In [30]:
R_square = r2_score(y_test, grid_predictions) 
print('Coefficient of Determination R2', R_square) 

Coefficient of Determination R2 0.9477670229749824


In [25]:
print("Model MAE", metrics.mean_absolute_error(y_test, grid_predictions))
print("Model MSE", metrics.mean_squared_error(y_test, grid_predictions))
print("Model RMSE",np.sqrt(metrics.mean_squared_error(y_test, grid_predictions)))

Model MAE 14.7782822035934
Model MSE 522.4042223899502
Model RMSE 22.85616377238206


In [26]:
x = [[2020.0	,1.0,	10.0,	11.75,	0.538194,	7.418056	,106.432118	,0.125000	,1.069097,	1.409028,	254.034722,	86.831493,	9.803819,	1023.920139]]

In [27]:
x = StandardScaler().fit_transform(x)

In [28]:
y_pred_test=grid.predict(x)
y_pred_test

array([227.74355389])